In [1]:
import essentia.standard as ess
import numpy as np
from scipy.signal import convolve
import glob, os, sys


In [2]:
import sox
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, AddImpulseResponse

In [3]:
sys.path.append('../../')

In [4]:
ir_path = './IR/'
store_path = '/media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/'

In [5]:
audio_path = '../../TFG_DATABASE_PROCESSED/'
raw_audios = glob.glob(audio_path + '*_DI.wav') #+ glob.glob(audio_path + '*_Reference.wav')
sample_rate = 44100
# audio = ess.MonoLoader(filename=audio_path, sampleRate=sample_rate)()

In [6]:
# Filter ir files

In [7]:
filtered_ir =['classroom','diningroom','livingroom','bedroom','office','kitchen']
not_wanted = ['conferenceroom','1txts','1txs','meetingroom','bathroom','hallway','largebrickwalledopenplanoffic','lobby','foyer']
ir_files = []
for f in glob.glob(ir_path + '*.wav'):
    f_norm = f.lower()
    if any(room in f_norm for room in filtered_ir):
        if any(room in f_norm for room in not_wanted ):
            pass
        else:
            ir_files.append(f)

In [8]:
ir_files

['./IR/h098_Bedroom_2txts.wav',
 './IR/h004_LivingRoom_Large_48txts.wav',
 './IR/h101_Classroom_2txts.wav',
 './IR/h033_Classroom_6txts.wav',
 './IR/h094_Campground_CabinLivingroom_2txts.wav',
 './IR/h002_Bedroom_62txts.wav',
 './IR/h027_Classroom_8txts.wav',
 './IR/h001_Bedroom_65txts.wav',
 './IR/h015_Bedroom_174txts.wav',
 './IR/h099_Classroom_2txts.wav',
 './IR/h006_Bedroom_42txts.wav',
 './IR/h106_Classroom_2txts.wav',
 './IR/h005_Office_Small_44txts.wav',
 './IR/h039_Classroom_5txts.wav',
 './IR/h045_Livingroom_4txts.wav',
 './IR/h064_Classroom_3txts.wav',
 './IR/h031_Bedroom_7txts.wav',
 './IR/h017_Livingroom_152txts.wav',
 './IR/h018_Kitchen_12txts.wav',
 './IR/h028_Classroom_8txts.wav',
 './IR/h065_Classroom_3txts.wav',
 './IR/h012_Kitchen_22txts.wav',
 './IR/h103_Classroom_2txts.wav',
 './IR/h037_Classroom_5txts.wav',
 './IR/h008_Bedroom_35txts.wav',
 './IR/h032_Bedroom_6txts.wav',
 './IR/h016_MasterBedroom_15txts.wav',
 './IR/h025_Diningroom_8txts.wav',
 './IR/h105_Classroom

In [9]:
reverb_names = [r.split('_')[0].replace(ir_path,'') for r in ir_files]

# APPLY EFFECTS

Chorus

In [9]:
tfm = sox.Transformer()
tfm.set_globals(guard = True)
tfm.chorus()

tfm.build_file(audio_path, store_path+ audio_path.split('/')[-1].replace('.wav','_') +'Chorus'+ '.wav')


True

Flanger

In [41]:
tfm = sox.Transformer()
tfm.set_globals(guard = True)
tfm.flanger()

tfm.build_file(audio_path,'./Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +'Flanger'+ '.wav')


True

Overdrive

In [42]:
tfm = sox.Transformer()
tfm.set_globals(guard = False)
tfm.overdrive()

tfm.build_file(audio_path,'./Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +'OverDrive'+ '.wav')


True

Phaser

In [43]:
tfm = sox.Transformer()
tfm.set_globals(guard = True)
tfm.phaser()

tfm.build_file(audio_path,'./Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +'Phaser'+ '.wav')


True

# APPLY FILTERS

High pass

In [52]:
freq = [200,300,400,500]
for f in freq:
    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.highpass(frequency = f)

    tfm.build_file(audio_path,'./Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +'Hpass'+str(f)+ '.wav')


Low pass

In [55]:
freq = [2000,2500,1000,500,300]
for f in freq:
    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.lowpass(frequency = f)

    tfm.build_file(audio_path,'./Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +'Lpass'+str(f)+ '.wav')


output_file: ./Reverbs/Century_Eastman_130_DI_Lpass2000.wav already exists and will be overwritten on build
output_file: ./Reverbs/Century_Eastman_130_DI_Lpass2500.wav already exists and will be overwritten on build
output_file: ./Reverbs/Century_Eastman_130_DI_Lpass1000.wav already exists and will be overwritten on build
output_file: ./Reverbs/Century_Eastman_130_DI_Lpass500.wav already exists and will be overwritten on build


Treble

In [56]:
boost = [-20,20]
for b in boost:
    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.treble(b)

    tfm.build_file(audio_path,'./Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +'Treble'+str(b)+ '.wav')


# APPLY REVERBERATIONS

In [11]:
count = 0
for aud in raw_audios:
    audio = ess.MonoLoader(filename=aud, sampleRate=sample_rate)()
    for ir,rev in zip(ir_files,reverb_names):
        ir_samples = list(ess.MonoLoader(filename=ir, sampleRate=sample_rate)())
        
        tfm = sox.Transformer()
        tfm.set_globals(guard = True) 
        tfm.fir(ir_samples)
        output_file_name = store_path + aud.split('/')[-1].replace('.wav','_') +rev + str(count) + '.wav'
        
        tfm.build_file(input_array = audio,
                        sample_rate_in = sample_rate,
                        output_filepath = output_file_name)
        count += 1
        # ess.MonoWriter(filename= './Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +rev.replace('.wav','') + '.wav' , format='wav')(audio_convolved)

# APPLY GAUSSIAN RANDOM NOISE

In [68]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np

NOISES = 5
SAMPLE_RATE = 441000

for n in range(0,NOISES):
    augment = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.1),])

    # Augment/transform/perturb the audio data
    augmented_samples = augment(samples=audio, sample_rate=SAMPLE_RATE)

    ess.MonoWriter(filename= './Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_')+'noise'+str(n)+'.wav', format='wav')(augmented_samples)